In [1]:
import torch
from transformers import DetrImageProcessor, DetrForSegmentation
from PIL import Image
import os
import matplotlib.pyplot as plt
import numpy as np
import cv2
import pulse2percept as p2p
from pulse2percept.implants import AlphaAMS
from pulse2percept.stimuli import ImageStimulus

/Users/nimrahs/miniconda3/envs/pulse2percept/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/nimrahs/miniconda3/envs/pulse2percept/lib/python3.11/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


In [2]:
# paths
input_video_dir = "/Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/Videos"
output_base_dir = "/Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSegNeSyAI/neSyAI_output"

os.makedirs(output_base_dir, exist_ok=True)

# symbolic AI rules
PERSON_CLASSES = {"person"}
VEHICLE_CLASSES = {"car", "bus", "truck", "motorcycle"}
MICRO_MOBILITY_CLASSES = {"bicycle"}
STATIC_HAZARD_CLASSES = { "bench", "chair", "fire hydrant", "traffic light"}

SYMBOLIC_ENCODING = {
    "background": 20,
    "person": 255,
    "vehicle_static": 210,
    "vehicle_moving": 230,
    "micro_static": 180,
    "micro_moving": 200,
    "static_hazard": 130
}

SEGMENT_COLORS = {
    "background": (0, 0, 0),
    "person": (0, 255, 0),
    "vehicle_static": (0, 0, 200),
    "vehicle_moving": (0, 0, 255),
    "micro_static": (255, 255, 0),
    "micro_moving": (255, 200, 0),
    "static_hazard": (255, 128, 0)
}

# DETR neural image segmentation model 
model_name = "facebook/detr-resnet-50-panoptic"
processor = DetrImageProcessor.from_pretrained(model_name)
seg_model = DetrForSegmentation.from_pretrained(model_name)
seg_model.eval()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
seg_model.to(device)

# pulse2percept implant
p2p_model = p2p.models.ScoreboardModel(xrange=(-7, 7), yrange=(-7, 7), xystep=0.1)
p2p_model.build()
implant = AlphaAMS()

def map_class_to_symbol(class_name, is_moving=False):
    if class_name in PERSON_CLASSES:
        return "person"
    elif class_name in VEHICLE_CLASSES:
        return "vehicle_moving" if is_moving else "vehicle_static"
    elif class_name in MICRO_MOBILITY_CLASSES:
        return "micro_moving" if is_moving else "micro_static"
    elif class_name in STATIC_HAZARD_CLASSES:
        return "static_hazard"
    else:
        return "background"

# Process each video
video_files = [f for f in os.listdir(input_video_dir) if f.endswith(".mp4")]

for video_file in video_files:
    video_path = os.path.join(input_video_dir, video_file)
    print(f"[INFO] Processing {video_file}")

    cap = cv2.VideoCapture(video_path)
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_idx = 0
    previous_seg_map = None

    # Create per-video output folders
    video_name = os.path.splitext(video_file)[0]
    maskDir = os.path.join(output_base_dir, video_name, "masks")
    perceptImgSeg = os.path.join(output_base_dir, video_name, "percepts")
    segVisDir = os.path.join(output_base_dir, video_name, "segmented")
    os.makedirs(maskDir, exist_ok=True)
    os.makedirs(perceptImgSeg, exist_ok=True)
    os.makedirs(segVisDir, exist_ok=True)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Convert to PIL image
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(frame_rgb)

        # Neural segmentation
        inputs = processor(images=image, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = seg_model(**inputs)

        pred_masks = outputs.pred_masks.squeeze(0).cpu().numpy()
        labels = outputs.logits.argmax(-1).squeeze(0).cpu().numpy()

        # Symbolic segmentation
        id2label = seg_model.config.id2label
        seg_map = np.zeros(pred_masks.shape[1:], dtype=np.uint8)

        for i, mask in enumerate(pred_masks):
            class_id = int(labels[i])
            if class_id >= len(id2label):
                continue
            class_name_label = id2label[class_id]
            if class_name_label == "no_object":
                continue

            # Motion detection
            is_moving = False
            if previous_seg_map is not None and np.any(mask > 0.5):
                prev_region = previous_seg_map[mask > 0.5]
                is_moving = np.std(prev_region) > 5

            symbol = map_class_to_symbol(class_name_label, is_moving)
            seg_map[mask > 0.5] = SYMBOLIC_ENCODING[symbol]

        previous_seg_map = seg_map.copy()

        # Save symbolic mask
        mask_path = os.path.join(maskDir, f"frame_{frame_idx:05d}.png")
        cv2.imwrite(mask_path, seg_map)

        # Segmentation visualization
        seg_color = np.zeros((*seg_map.shape, 3), dtype=np.uint8)
        for k, v in SYMBOLIC_ENCODING.items():
            seg_color[seg_map == v] = SEGMENT_COLORS[k]
        seg_vis_path = os.path.join(segVisDir, f"frame_{frame_idx:05d}.png")
        cv2.imwrite(seg_vis_path, seg_color)

        # pulse2percept
        stimulus = p2p.stimuli.ImageStimulus(seg_vis_path)
        stimulus = stimulus.resize(implant.shape)
        implant.stim = stimulus
        percept = p2p_model.predict_percept(implant)
        percept_path = os.path.join(perceptImgSeg, f"frame_{frame_idx:05d}.png")
        percept.save(percept_path)

        frame_idx += 1

    cap.release()

    # recombine frame to video
    percept_files = sorted(os.listdir(perceptImgSeg))
    if len(percept_files) == 0:
        print(f"[WARNING] No frames for {video_file}, skipping video creation")
        continue

    first_frame = cv2.imread(os.path.join(perceptImgSeg, percept_files[0]))
    height, width, _ = first_frame.shape
    output_video_path = os.path.join(output_base_dir, f"{video_name}_percept.mp4")

    out = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))
    for f in percept_files:
        frame = cv2.imread(os.path.join(perceptImgSeg, f))
        out.write(frame)
    out.release()

    print(f"Percept video saved to: {output_video_path}")

print("All videos processed!")


Some weights of the model checkpoint at facebook/detr-resnet-50-panoptic were not used when initializing DetrForSegmentation: ['detr.model.backbone.conv_encoder.model.layer1.0.downsample.1.num_batches_tracked', 'detr.model.backbone.conv_encoder.model.layer2.0.downsample.1.num_batches_tracked', 'detr.model.backbone.conv_encoder.model.layer3.0.downsample.1.num_batches_tracked', 'detr.model.backbone.conv_encoder.model.layer4.0.downsample.1.num_batches_tracked']
- This IS expected if you are initializing DetrForSegmentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DetrForSegmentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[INFO] Processing North-East.mp4


Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/North-East/percepts/frame_00000.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/North-East/percepts/frame_00001.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/North-East/percepts/frame_00002.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/North-East/percepts/frame_00003.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/North-East/percepts/frame_00004.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/North-East/percepts/frame_00005.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/North-East/percepts/frame_00006.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/North-East/percepts/frame_00007.png.
Created /Users/n

[INFO] Percept video saved to: /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/North-East_percept.mp4
[INFO] Processing road-video-dubai.mp4


Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/road-video-dubai/percepts/frame_00000.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/road-video-dubai/percepts/frame_00001.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/road-video-dubai/percepts/frame_00002.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/road-video-dubai/percepts/frame_00003.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/road-video-dubai/percepts/frame_00004.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/road-video-dubai/percepts/frame_00005.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/road-video-dubai/percepts/frame_00006.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/road-video-

...

[INFO] Percept video saved to: /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/pplWalk1_percept.mp4
[INFO] Processing pplWalk2.mp4


Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/pplWalk2/percepts/frame_00000.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/pplWalk2/percepts/frame_00001.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/pplWalk2/percepts/frame_00002.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/pplWalk2/percepts/frame_00003.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/pplWalk2/percepts/frame_00004.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/pplWalk2/percepts/frame_00005.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/pplWalk2/percepts/frame_00006.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/pplWalk2/percepts/frame_00007.png.
Created /Users/nimrahs/Desktop/S

[INFO] Percept video saved to: /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/pplWalk2_percept.mp4
[INFO] Processing night1.mp4


Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/night1/percepts/frame_00000.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/night1/percepts/frame_00001.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/night1/percepts/frame_00002.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/night1/percepts/frame_00003.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/night1/percepts/frame_00004.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/night1/percepts/frame_00005.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/night1/percepts/frame_00006.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/night1/percepts/frame_00007.png.
Created /Users/nimrahs/Desktop/SaleemP2P/Officia

[INFO] Percept video saved to: /Users/nimrahs/Desktop/SaleemP2P/Official/FINAL_ImageSeg/NeSyAI/neSyAI_output/night1_percept.mp4
[INFO] All videos processed!
